In [1]:
import os, sys
sys.path.append('../src')
sys.path.append('../src/utils/')
from spiking_dataloader import WISDM_spiking_dataloader, WisdmDatasetParser
from inibitory_network.inibitory_network_h1_test import InibitoryLifNet
from output_process import OutputProcess

In [2]:
dataset = WisdmDatasetParser('../data/data_watch_subset_0_40.npz', norm=None, class_sublset='custom', subset_list=[0, 2, 6, 8, 9, 14, 17])

(6,)
(6,)
ytrain shape (55404, 18)
yval shape (18468, 18)
ytest shape (18469, 18)
num classes train dataset: 7 occurrences of each class:[3127 2855 3102 3047 3150 2973 3213]
num classes eval dataset: 7 occurrences of each class:[1035  935 1122  996 1110 1007 1077]
num classes test dataset: 7 occurrences of each class:[1046  900 1046 1036 1076  982 1054]


In [3]:
num_samples = 100 

In [4]:
val_set = dataset.get_validation_set(subset=num_samples, shuffle=False)
val_set[0].shape

(100, 6, 40)

In [5]:
spiking_loader = WISDM_spiking_dataloader(val_set ,clear_intervall=40)

In [6]:
net = InibitoryLifNet('/home/roboto/snntorch_network/nni_experiments/Inibitory_lif_no_encoder/results/hjulef4s/trials/RHEsE/Trained/network_best.npz')
net.print_state_variables()

leaky1 betas:0.1827973872423172
leaky1 vth:1.711238145828247
leaky2 betas:0.8350825309753418
leaky2 vth:0.5812320113182068
recurrent betas:0.8625779747962952
leaky3 betas:[1.]
leaky3 vth:1.5450438261032104
recurrent beta:0.8625779747962952
###network state variables###
leaky1_betas: 0.8172026127576828
leaky1_vth: 1.711238145828247
leaky2_betas: 0.1649174690246582
leaky2_vth: 0.5812320113182068
leaky3_betas: [0.]
leaky3_vth: 1.5450438261032104



In [7]:
net.print_weights()

###network weights###
linear1_w: [[ 1.07709523e-02 -1.23940095e-01 -2.68006008e-02 -4.61998731e-02  6.81097209e-02 -1.53923154e-01]
 [-5.24332896e-02 -1.44148067e-01  3.03932913e-02 -1.95165977e-01 -1.14604183e-01 -3.01601551e-02]
 [-2.28118375e-02  3.47479954e-02  1.44239411e-01  9.29336071e-01 -5.17672002e-01 -1.26216650e+00]
 [-3.56787886e-03  1.09616093e-01 -1.16333283e-01  2.00987142e-02  7.31353536e-02  2.71097630e-01]
 [-1.04006797e-01 -2.86299214e-02  5.45209497e-02  4.39049080e-02 -3.31520773e-02  3.48328203e-01]
 [ 1.24379382e-01 -5.96372597e-02  1.33920321e-02 -5.61634228e-02  1.26217902e+00 -1.83073264e-02]
 [-1.53475618e-02  2.83372831e-02  8.28295499e-02 -7.41664171e-01 -6.02275208e-02 -7.27674812e-02]
 [-6.49568513e-02  1.35022134e-01 -1.74002901e-01 -1.02230035e-01  7.18384385e-01  3.03249568e-01]
 [-7.62214661e-02 -5.63413016e-02  4.84496877e-02  8.56221244e-02  4.53585573e-02  2.78760105e-01]
 [ 6.51120991e-02  7.23755956e-02 -1.43787831e-01 -7.81078637e-02  2.1873445

In [8]:
out_sink = OutputProcess(7,num_samples,80, 0)

In [9]:
spiking_loader.data_out.connect(net.a_in)
net.s_out.connect(out_sink.spikes_in)
out_sink.label_in.connect_from(spiking_loader.label_out)

In [10]:

from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg, Loihi2SimCfg
import numpy as np
from tqdm import tqdm

for i in tqdm(range(num_samples)):
        out_sink.run(condition=RunSteps(num_steps=80),
                        run_cfg=Loihi2SimCfg(select_sub_proc_model=True,
                        select_tag='floating_pt'))
        net.reset_hidden_state()


ground_truth = out_sink.gt_labels.get().astype(np.int32)
predictions = out_sink.pred_labels.get().astype(np.int32)

# Stop the execution
out_sink.stop()

accuracy = np.sum(ground_truth==predictions)/ground_truth.size * 100

print(f"\nGround truth: {ground_truth}\n"
      f"Predictions : {predictions}\n"
      f"Accuracy    : {accuracy}")


  0%|          | 0/100 [00:00<?, ?it/s]

leaky1 dv:1
leaky1 du:0.8172026127576828
leaky1 vth:1.711238145828247
leaky2 dv:0.1649174690246582
leaky2 du:1
leaky2 vth:0.5812320113182068
recurrent du:0.13742202520370483
leaky3 dv:[0.]
leaky3 du:1
leaky3 vth:1.5450438261032104


100%|██████████| 100/100 [01:35<00:00,  1.05it/s]


Ground truth: [2 4 0 3 5 0 0 0 3 3 1 5 0 4 2 1 5 1 6 6 6 0 4 6 4 0 4 5 3 2 2 6 0 3 1 1 5 4 2 6 4 6 3 1 5 0 5 4 2 2 6 0 2 6 2 1 2 4 1 5 2 5 3 3 5 1 1 4 4 6 4 4 2 0 1 3 0 4 6 0 2 4 4 1 5 6 5 5 6 0 0 6 0 0 0 3 5 1 0 1]
Predictions : [2 3 0 3 6 0 0 0 3 2 0 6 0 4 2 0 6 1 6 6 6 0 3 3 4 0 3 6 3 2 2 3 1 3 0 0 6 3 2 6 4 4 3 0 6 0 4 4 2 2 3 0 2 1 1 0 2 4 0 1 2 6 3 3 6 0 0 4 4 1 4 4 2 0 0 3 0 4 6 0 2 4 3 2 6 1 1 6 6 0 0 6 0 0 0 3 6 0 0 0]
Accuracy    : 57.99999999999999
